# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re
from functools import wraps
from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        arg_str = ', '.join(repr(arg) for arg in args)
        if arg_str!='':
          print(f'{func.__name__}({arg_str}) потребовалось: {end - start} секунд')
        else:
          print(f'{func.__name__} потребовалось: {end - start} секунд')
        return result
    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'Функция {func.__name__} вызвана с параметрами:\n {args}, {kwargs}')
        return result
    return wrapper

## Задание 3

In [5]:
c = 0
def counter(func):

    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        global c
        c += 1
        result = func(*args, **kwargs)
        print(f'Функция вызвана: {c} раз')
        return result
    return wrapper

## Задание 4

In [6]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}
  @wraps(func)
  def fmemo(*args):
    if args in cache:
        return cache[args]
    else:
        cache[args] = func(*args)
        return cache[args]

#   memo.cache = cache
  return fmemo

## Тестирование

In [14]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

word_count('whole') потребовалось: 0.4393630027770996 секунд
Функция word_count вызвана с параметрами:
 ('whole',), {}
Функция вызвана: 3 раз
Cлово whole встречается 176 раз


In [8]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [11]:
# измеряем время выполнения обычной функции
%%timeit
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)
fib(30)

200 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
# измеряем время выполнения функции с декоратором @memo который кэширует данные
%%timeit
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)
fib(30)

15.1 µs ± 3.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
